In [1]:
# imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
df = pd.read_csv("../data/train_sample.csv.zip")

# Density

## Group by IP/Device/OS 

Again, if it's a public hotspot, we may see some spree behavior here. If the # of clicks from this triplet is low, but heavily concentrated ... might be some fraud.

In [3]:
df.click_time = pd.to_datetime(df.click_time)
df['time_diff_triplet'] = df.sort_values(['ip','device','os','click_time']).groupby(['ip','device','os'])['click_time'].diff()


In [4]:
def get_seconds(time_delta):
    return time_delta.seconds

df['seconds_triplet'] = df['time_diff_triplet'].apply(get_seconds)

### Sanity Check - do the seconds match the time_diff?

For this ip/device/os ... 8 hours between 1st and 2nd click, 3 hours between 2nd and 3rd click, and so on. 

In [5]:
df[(df.ip==118315) & (df.device==1) & (df.os==13)].sort_values(['click_time'])

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,time_diff_triplet,seconds_triplet
66615,118315,1,1,13,137,2017-11-07 08:36:03,NaN,0,NaT,NaN
28804,118315,3,1,13,280,2017-11-07 12:42:50,NaN,0,04:06:47,14807.0
62507,118315,18,1,13,134,2017-11-07 14:21:56,NaN,0,01:39:06,5946.0
36702,118315,3,1,13,489,2017-11-08 05:12:58,NaN,0,14:51:02,53462.0
64422,118315,6,1,13,459,2017-11-08 10:46:43,NaN,0,05:33:45,20025.0
66607,118315,15,1,13,245,2017-11-08 16:08:30,NaN,0,05:21:47,19307.0
89984,118315,11,1,13,137,2017-11-09 01:12:29,NaN,0,09:03:59,32639.0
68039,118315,2,1,13,237,2017-11-09 02:52:44,NaN,0,01:40:15,6015.0
18347,118315,2,1,13,237,2017-11-09 03:05:28,NaN,0,00:12:44,764.0
94144,118315,2,1,13,237,2017-11-09 05:21:52,NaN,0,02:16:24,8184.0


In [6]:
print(df.seconds_triplet.describe())
df.seconds_triplet.hist()

count    22384.000000
mean     28121.416011
std      24946.066455
min          0.000000
25%       6364.750000
50%      20706.500000
75%      45335.500000
max      86397.000000
Name: seconds_triplet, dtype: float64


ImportError: No module named matplotlib.pyplot

    ### Wouldnt a robot user be fast?

For spree clicks, i would expect very little time between clicks. Let's filter seconds to < 1 minute and see how many < 1 minute lags there are for these triplets.

In [ ]:
print(df[df.seconds_triplet < 60]['seconds_triplet'].describe())
df[df.seconds_triplet < 60]['seconds_triplet'].hist()

In [ ]:
df[df.seconds_triplet<60].groupby(['ip','device','os']).size().sort_values(ascending=False)

### 5314 - is the public hotspot

In [ ]:
df[(df.seconds_triplet <60) & (df.ip == 5314)].sort_values(['click_time'])

## Same analysis, Groupby IP (instead of triplets)

just to see the distribution. As expected, presumably the hotspots are at the top.

In [ ]:
df['time_diff_ip'] = df.sort_values(['ip','click_time']).groupby(['ip'])['click_time'].diff()
df['seconds_ip'] = df['time_diff_ip'].apply(get_seconds)

In [ ]:
print(df[df.seconds_ip < 60]['seconds_ip'].describe())
df[df.seconds_ip < 60]['seconds_ip'].hist()

In [ ]:
df[df.seconds_ip<60].groupby(['ip']).size().sort_values(ascending=False).to_frame('count') 

In [ ]:
df[(df.seconds_ip <60) & (df.ip == 5348)].sort_values(['click_time']) # 5348 a hotspot?